# p-median

## 필요한 모듈 설치 및 라이브러리 불러오기

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
!pip install pulp
!pip install haversine

In [4]:
# data handling
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings(action = "ignore")

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 좌표 단위 변환
from functools import partial
from pyproj import Proj, transform

# p-median
from pulp import *

In [15]:
shelter = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/P-Median-Algorithm/data/수해대피소.csv")
people = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/P-Median-Algorithm/data/기초생활수급자_독산1동.csv")

In [16]:
people_lat = people['Y']
people_long = people['X']

shelter_lat = shelter['Y']
shelter_long = shelter['X']

In [17]:
from haversine import haversine


- 가중치 아이디어(1 ~ 1.5) -> 비슷한 거리에 있을 때 더 수용 가능한 곳으로 가도록 하게 하기 위해
  - shelter['weight'][j] : j번째 시설의 해당 weight (학교가 가장 대피하기 좋은 장소(수용인원이 많음) 기준 1로 두고 나머지 유형의 대피소의 가중치는 학교 수용인원수 / 해당 유형의 대피소의 수용인원수)
  - minmaxscaling 후 2로 나누고 1을 더하기

In [18]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()
shelter["weight"] = mm.fit_transform(shelter[["weight"]])

for j in range(shelter.shape[0]):
    shelter["weight"][j] = shelter["weight"][j]/2 + 1

c = []
for i in range(people.shape[0]):
    list_ = []
    for j in range(shelter.shape[0]):
        start = (float(people_lat[i]), float(people_long[i]))
        goal = (float(shelter_lat[j]), float(shelter_long[j]))
        list_.append(haversine(start, goal) * (shelter["weight"][j]))
    c.append(list_)

- c : 각 시설별로 각 고객에게 서비스를 제공하는 비용(cost함수 정의 후 만들기)
- i행은 고객을 나타냄
- j열은 시설을 나타냄
- c[i][j] : i번째 고객에게 j번째 facility가 서비스를 제공하는 비용

- m : 고객의 수(기초수급자의 수 : 1306)
- n : 시설의 수(공공기관의 수 : 111)
- p : 찾을 시설의 수(면적으로 나누기)

In [25]:
m = 1306
n = 111
p = 6

# 문제 정의
prob = LpProblem("p-median", LpMinimize)

# 결정 변수 정의
x = LpVariable.dicts("x", [(i,j) for i in range(m) for j in range(n)], 0, 1, LpBinary)
y = LpVariable.dicts("y", [j for j in range(n)], 0, 1, LpBinary)

- x는 이진 결정 변수의 dictionary를 정의
    - 각 변수는 i고객이 j시설에 할당되었는지 여부를 나타냄
    
- 각각의 시설이 열렸는지 닫혔는지?

In [26]:
# 목적함수 정의
prob += lpSum([c[i][j] * x[(i,j)] for i in range(m) for j in range(n)])

In [27]:
for i in range(m):
    prob += lpSum([x[(i,j)] for j in range(n)]) == 1
# 한 사람은(i번째 사람) 한 시설(j번째 시설)에만 영향을 받아야 함

for i in range(m):
    for j in range(n):
        prob += x[(i,j)] <= y[j]
# 시설이 열려 있어야지 영향을 받고 있는지 여부를 0과 1로 결정할 수 있음

prob += lpSum([y[j] for j in range(n)]) == p
# 시설이 열려있는 수는 우리가 원하는 시설의 수와 같아야됨

In [28]:
# branch-and-bound algorithm을 이용해 문제 해결하기
random.seed(123)
prob.solve(PULP_CBC_CMD(gapRel=0.0, threads=1, timeLimit=120))

1

- gapRel = 0.0 : 주어진 시간 제한 내에서 최상의 솔루션을 찾을 때까지 최적의 솔루션을 계속 검색
- threads = 1 : 병렬 컴퓨팅에 사용할 수 있는 thread 수를 의미, 단일 thread 모들에서 실행한다는 의미인 1로 실행
- timeLimit = 600 : 솔루션을 찾는 데 사용할 수 있는 최대 시간(초)

In [29]:
# Print results
print("Optimal objective value:", value(prob.objective))
for j in range(n):
    if y[j].value() > 0.5:
        print("Facility", j, "is located.")
        for i in range(m):
                if x[(i,j)].value() > 0.5:
                    print("- Customer", i, "is served.")

Optimal objective value: 344.09722953252765
Facility 14 is located.
- Customer 4 is served.
- Customer 9 is served.
- Customer 19 is served.
- Customer 23 is served.
- Customer 26 is served.
- Customer 31 is served.
- Customer 87 is served.
- Customer 89 is served.
- Customer 92 is served.
- Customer 99 is served.
- Customer 100 is served.
- Customer 125 is served.
- Customer 130 is served.
- Customer 135 is served.
- Customer 148 is served.
- Customer 152 is served.
- Customer 155 is served.
- Customer 163 is served.
- Customer 172 is served.
- Customer 173 is served.
- Customer 176 is served.
- Customer 177 is served.
- Customer 178 is served.
- Customer 179 is served.
- Customer 189 is served.
- Customer 236 is served.
- Customer 264 is served.
- Customer 271 is served.
- Customer 273 is served.
- Customer 290 is served.
- Customer 301 is served.
- Customer 307 is served.
- Customer 312 is served.
- Customer 313 is served.
- Customer 315 is served.
- Customer 319 is served.
- Custom

- 첫 번째 줄 : p개 시설을 open했을 때 최소 비용을 인쇄(목표 값)
- 그 다음 각각 시설에 대해 코드가 시설이 열렸는지 여부를 확인
    - y[j]의 값이 0.5보다 크면 j번째 시설이 열렸다는 의미

In [32]:
import folium
lat = shelter['Y'].mean()
long = shelter['X'].mean()

#지도 띄우기
m = folium.Map([lat, long], zoom_start = 20)

#for n in range(shelter.shape[0]):
for n in (14, 19, 23, 30, 60, 107):
    #name = df_seoul_hospital.loc[n, "상호명"]
    #address = df_seoul_hospital.loc[n, "도로명주소"]
    popup = shelter.loc[n, 'shelter'],
    location = [shelter.loc[n, "Y"], shelter.loc[n, "X"]]
    folium.Marker(
        location = location,
        popup = popup
    ).add_to(m)
m

In [33]:
df_result = shelter.loc[[14, 19, 23, 30, 60, 107]]
df_result

,shelter,X,Y,weight
14,세일중학교,126.895601,37.474627,1.0
19,가산중학교,126.894412,37.468445,1.0
23,서울안천초등학교,126.888492,37.458168,1.0
30,서울두산초등학교,126.890988,37.467144,1.0
60,경로당17,126.895287,37.464589,1.5
107,경로당64,126.892150,37.461081,1.5


## 기대효과 분석
- 기존 수해대피소에 비해 비용이 얼마나 감소하였는가?

In [35]:
shelter_located = pd.DataFrame({"shelter" : ["가산중학교", "가산초등학교", "시흥초등학교", "안천중학교"],
                         "X" : [126.894456, 126.896308, 126.904625, 126.887484],
                         "Y" : [37.46876, 37.4781501, 37.4553258, 37.4595092],
                          "weight" : [1, 1, 1, 1]})

In [36]:
from sklearn.preprocessing import MinMaxScaler

people_lat = people['Y']
people_long = people['X']

shelter11_lat = shelter['Y']
shelter11_long = shelter['X']

mm = MinMaxScaler()
shelter_located["weight"] = mm.fit_transform(shelter_located[["weight"]])

for j in range(shelter_located.shape[0]):
    shelter_located["weight"][j] = shelter_located["weight"][j]/2 + 1

c11 = []
for i in range(people.shape[0]):
    list_ = []
    for j in range(shelter_located.shape[0]):
        start = (float(people_lat[i]), float(people_long[i]))
        goal = (float(shelter11_lat[j]), float(shelter11_long[j]))
        list_.append(haversine(start, goal) * (shelter_located["weight"][j]))
    c11.append(list_)

In [37]:
m = 1306
n = 4
p = 4

# 문제 정의
prob_new = LpProblem("p-median", LpMinimize)

# 결정 변수 정의
x = LpVariable.dicts("x", [(i,j) for i in range(m) for j in range(n)], 0, 1, LpBinary)
y = LpVariable.dicts("y", [j for j in range(n)], 0, 1, LpBinary)

In [38]:
# 목적함수 정의
prob_new += lpSum([c11[i][j] * x[(i,j)] for i in range(m) for j in range(n)])

In [39]:
for i in range(m):
    prob_new += lpSum([x[(i,j)] for j in range(n)]) == 1
# 한 사람은(i번째 사람) 한 시설(j번째 시설)에만 영향을 받아야 함

for i in range(m):
    for j in range(n):
        prob_new += x[(i,j)] <= y[j]
# 시설이 열려 있어야지 영향을 받고 있는지 여부를 0과 1로 결정할 수 있음

prob_new += lpSum([y[j] for j in range(n)]) == p
# 시설이 열려있는 수는 우리가 원하는 시설의 수와 같아야됨

In [40]:
# branch-and-bound algorithm을 이용해 문제 해결하기
random.seed(123)
prob_new.solve(PULP_CBC_CMD(gapRel=0.0, threads=1, timeLimit=120))

1

In [41]:
# Print results
print("Optimal objective value:", value(prob_new.objective))
for j in range(n):
    if y[j].value() > 0.5:
        print("Facility", j, "is located.")
#        for i in range(m):
#                if x[(i,j)].value() > 0.5:
#                    print("- Customer", i, "is served.")

Optimal objective value: 981.1411506510686
Facility 0 is located.
Facility 1 is located.
Facility 2 is located.
Facility 3 is located.


In [43]:
import folium
lat = shelter_located['Y'].mean()
long = shelter_located['X'].mean()

#지도 띄우기
m = folium.Map([lat, long], zoom_start = 20)

#for n in range(shelter.shape[0]):
for n in (0, 1, 2, 3):
    #name = df_seoul_hospital.loc[n, "상호명"]
    #address = df_seoul_hospital.loc[n, "도로명주소"]
    popup = shelter_located.loc[n, 'shelter'],
    location = [shelter_located.loc[n, "Y"], shelter_located.loc[n, "X"]]
    folium.Marker(
        location = location,
        popup = popup
    ).add_to(m)
m

In [44]:
shelter_located

,shelter,X,Y,weight
0,가산중학교,126.894456,37.468760,1.0
1,가산초등학교,126.896308,37.478150,1.0
2,시흥초등학교,126.904625,37.455326,1.0
3,안천중학교,126.887484,37.459509,1.0
